#def table_floods_ems(ems_url: str, hazard_type: List[str]) -> pandas.core.frame.DataFrame:
def table_floods_ems():
    """
    This function reads the list of EMS Rapid Mapping Activations 
    from the Copernicus Emergency Management System for Event type
    'Flood' and 'Storm' and returns a pandas.DataFrame of flood events.

    Args:
      None
    Returns:
     A pandas.DataFrame of Flood events. 

    """
    # FIXME: Why are the inputs hard-coded?!!!!
    # lines 28 - 30 exist in copernicusEMS_web_crawler.py
    ems_web_page = "https://emergency.copernicus.eu/mapping/list-of-activations-rapid"
    tables = pd.read_html(ems_web_page)[0]
    tables_floods = tables[(tables.Type == "Flood") | (tables.Type == "Storm")]
    tables_floods = tables_floods[tables_floods["Event Date"] >= "2014-05-01"]
    tables_floods = tables_floods.reset_index()[['Act. Code', 'Title', 'Event Date', 'Type', 'Country/Terr.']]
    tables_floods = tables_floods.rename({"Act. Code": "Code", "Country/Terr.": "Country", "Event Date": "CodeDate"},
                                         axis=1)
    tables_floods = tables_floods.set_index("Code")

    return tables_floods

In [2]:
from requests_html import HTMLSession
import pandas as pd
import requests
import os
from webbot import Browser
import time
import shutil
import logging
from typing import List

In [3]:
def is_downloadable(url: str) -> bool:
    """ 
    Function that checks if the url contains a downloadable resource

    Args:
      url:
        A string containing the Copernicus EMS url grabbed
        for specific attributes.

    Returns:
      A boolean indicating if the url is valid
    """

    h = requests.head(url, allow_redirects=True)
    header = h.headers
    content_type = header.get('content-type').lower()
    return ('text' not in content_type or 'html' not in content_type)

In [4]:
#url = 'https://emergency.copernicus.eu/mapping/list-of-activations-rapid'
url = 'https://emergency.copernicus.eu/mapping/download/184574/EMSR497_AOI01_DEL_MONIT06_r1_RTP01_v1_vector.zip'
h = requests.head(url, allow_redirects=True)
header = h.headers
content_type = header.get('content-type').lower()

print("header: ", header)
print("h.headers: ", h.headers)
print(content_type)
print('text' not in content_type or 'html' not in content_type)

header:  {'Date': 'Tue, 16 Feb 2021 20:38:33 GMT', 'Server': 'Apache', 'Content-Security-Policy': 'upgrade-insecure-requests;', 'X-Drupal-Cache': 'MISS', 'Expires': 'Sun, 19 Nov 1978 05:00:00 GMT', 'Last-Modified': 'Tue, 16 Feb 2021 20:38:33 +0000', 'Cache-Control': 'public, max-age=0', 'ETag': '"1613507913-1"', 'Content-Language': 'en', 'Vary': 'Cookie,Accept-Encoding', 'Content-Encoding': 'gzip', 'Keep-Alive': 'timeout=15, max=256', 'Connection': 'Keep-Alive', 'Content-Type': 'text/html; charset=utf-8'}
h.headers:  {'Date': 'Tue, 16 Feb 2021 20:38:33 GMT', 'Server': 'Apache', 'Content-Security-Policy': 'upgrade-insecure-requests;', 'X-Drupal-Cache': 'MISS', 'Expires': 'Sun, 19 Nov 1978 05:00:00 GMT', 'Last-Modified': 'Tue, 16 Feb 2021 20:38:33 +0000', 'Cache-Control': 'public, max-age=0', 'ETag': '"1613507913-1"', 'Content-Language': 'en', 'Vary': 'Cookie,Accept-Encoding', 'Content-Encoding': 'gzip', 'Keep-Alive': 'timeout=15, max=256', 'Connection': 'Keep-Alive', 'Content-Type': 'te

In [5]:
def table_floods_ems():
    """
    This function reads the list of EMS Rapid Mapping Activations 
    from the Copernicus Emergency Management System for Event type
    'Flood' and 'Storm' and returns a pandas.DataFrame of flood events.

    Args:
      None
    Returns:
     A pandas.DataFrame of Flood events. 

    """
    # FIXME: Why are the inputs hard-coded?!!!!
    # lines 28 - 30 exist in copernicusEMS_web_crawler.py
    ems_web_page = "https://emergency.copernicus.eu/mapping/list-of-activations-rapid"
    tables = pd.read_html(ems_web_page)[0]
    tables_floods = tables[(tables.Type == "Flood") | (tables.Type == "Storm")]
    tables_floods = tables_floods[tables_floods["Event Date"] >= "2014-05-01"]
    tables_floods = tables_floods.reset_index()[['Act. Code', 'Title', 'Event Date', 'Type', 'Country/Terr.']]
    tables_floods = tables_floods.rename({"Act. Code": "Code", "Country/Terr.": "Country", "Event Date": "CodeDate"},
                                         axis=1)
    #tables_floods = tables_floods.set_index("Code")

    return tables_floods

In [6]:
ems_web_page = "https://emergency.copernicus.eu/mapping/list-of-activations-rapid"
tf = table_floods_ems()
tf

,Code,Title,CodeDate,Type,Country
0,EMSR501,Flood in Albania,2021-01-06,Flood,Albania
1,EMSR498,"Flood in Corrèze department, France",2021-02-02,Flood,France
2,EMSR497,Flood in Germany,2021-02-01,Flood,Germany
3,EMSR496,"Flood in Lazio Region, Italy",2021-01-26,Flood,Italy
4,EMSR495,"Tropical cyclone Eloise in Mozambique,...",2021-01-22,Storm,"Mozambique, Eswatini, Zimbabwe"
...,...,...,...,...,...
183,EMSR087,Floods in Bosnia and Herzegovina,2014-05-13,Flood,"Bosnia and Herze..., Serbia"
184,EMSR086,Floods in Serbia,2014-05-15,Flood,Serbia
185,EMSR085,"LuModExercise - Flood in Mertert,...",2014-05-07,Flood,Luxembourg
186,EMSR083,"Floods in Marche, Italy",2014-05-03,Flood,Italy


In [7]:
tf.dtypes

Code        object
Title       object
CodeDate    object
Type        object
Country     object
dtype: object

In [8]:
tf["Code"] = tf["Code"].astype(str)
tf.Code[0]

'EMSR501'

In [9]:
def fetch_copernicusEMS(ems_webpage: str, disaster_type: List[str], start_date: str):
    """
    This function takes in the copernicus emergency management system url
    and parses the list of activations for a disaster_type.
    
    Args:
      ems_webpage (str):
        A string describing the url for copernicus EMS.
      disaster_type (List[str]):
        A list of strings describing the disaster type to used to filter 
        events from the table of emergency activations.
      start_date (str):
        A string in 'YYY-MM-DD' describing the starting date for the gathered 
        events. The events will be filtered from that day to the present.
        
    Returns:
      tables_floods:
        A pandas DataFrame containing the flood events 
    
    """
    tables = pd.read_html(ems_web_page)[0]
    tables_floods = tables[tables["Type"].isin(disaster_type)]
    tables_floods = tables_floods[tables_floods["Event Date"] >= start_date]
#     tables_floods = tables_floods.reset_index()[['Act. Code', 'Title', 'Event Date', 'Type', 'Country/Terr.']]
#     tables_floods = tables_floods.rename({"Act. Code": "Code", "Country/Terr.": "Country", "Event Date": "CodeDate"},
#                                          axis=1)
#     tables_floods = tables_floods.set_index("Code")
    
    return tables_floods

In [10]:
ems_web_page = "https://emergency.copernicus.eu/mapping/list-of-activations-rapid"
tables_floods = fetch_copernicusEMS(ems_web_page, ["Flood", "Storm"], "2021-01-06")

In [11]:
tables_floods

,Unnamed: 0,Act. Code,Title,Event Date,Type,Country/Terr.,Feed
0,NaN,EMSR501,Flood in Albania,2021-01-06,Flood,Albania,NaN
2,NaN,EMSR498,"Flood in Corrèze department, France",2021-02-02,Flood,France,NaN
3,NaN,EMSR497,Flood in Germany,2021-02-01,Flood,Germany,NaN
4,NaN,EMSR496,"Flood in Lazio Region, Italy",2021-01-26,Flood,Italy,NaN
5,NaN,EMSR495,"Tropical cyclone Eloise in Mozambique,...",2021-01-22,Storm,"Mozambique, Eswatini, Zimbabwe",NaN


In [12]:
t_2 = tables_floods['Act. Code'].tolist()
t_2

['EMSR501', 'EMSR498', 'EMSR497', 'EMSR496', 'EMSR495']

In [13]:
tables_floods.shape

(5, 7)

In [14]:
for i, t in enumerate(tables_floods.head().itertuples()):
    print(t._2)

EMSR501
EMSR498
EMSR497
EMSR496
EMSR495


In [25]:
# FIXME!!!!!!
def download_vector_cems(url_zip_file, download_path, folder_out="CopernicusEMS"):
    """
    
    """
    name_zip = os.path.basename(url_zip_file)

    if not os.path.exists(folder_out):
        os.mkdir(folder_out) # creates a directory called CopernicusEMS to hold data

    file_path_out = os.path.join(folder_out, name_zip)
    if os.path.exists(file_path_out):
        logging.info("\tFile %s exists will not be download"%url_zip_file)
        return

    if is_downloadable(url_zip_file):
        r = requests.get(url_zip_file, allow_redirects=True)
        open(file_path_out, 'wb').write(r.content)
    else:
        web = Browser(showWindow=True)
        web.go_to(url_zip_file)
        time.sleep(1)
        web.click("Confirmation")
        time.sleep(1)
        web.press(web.Key.ENTER)
        time.sleep(1)
        expected_file = os.path.join(download_path, name_zip)
        for _i in range(300):
            if os.path.exists(expected_file):
                shutil.move(expected_file, file_path_out)
                break
            elif (_i % 10) == 0:
                logging.info("\t%ds File %s not downloaded yet"%(_i, expected_file))
            time.sleep(1)

        web.close_current_tab()

    if not os.path.exists(file_path_out):
        logging.warning("\tFAIL in download file %s" % url_zip_file)

In [22]:
print(is_downloadable('https://emergency.copernicus.eu/mapping/download/184574/EMSR497_AOI01_DEL_MONIT06_r1_RTP01_v1_vector.zip'))

False


In [15]:
w = tf[:]
w

,Code,Title,CodeDate,Type,Country
0,EMSR501,Flood in Albania,2021-01-06,Flood,Albania
1,EMSR498,"Flood in Corrèze department, France",2021-02-02,Flood,France
2,EMSR497,Flood in Germany,2021-02-01,Flood,Germany
3,EMSR496,"Flood in Lazio Region, Italy",2021-01-26,Flood,Italy
4,EMSR495,"Tropical cyclone Eloise in Mozambique,...",2021-01-22,Storm,"Mozambique, Eswatini, Zimbabwe"
...,...,...,...,...,...
183,EMSR087,Floods in Bosnia and Herzegovina,2014-05-13,Flood,"Bosnia and Herze..., Serbia"
184,EMSR086,Floods in Serbia,2014-05-15,Flood,Serbia
185,EMSR085,"LuModExercise - Flood in Mertert,...",2014-05-07,Flood,Luxembourg
186,EMSR083,"Floods in Marche, Italy",2014-05-03,Flood,Italy


In [16]:
zip_urls = []
for code in tf['Code']:
    product_url = "https://emergency.copernicus.eu/mapping/list-of-components/" + code
    session = HTMLSession()
    r = session.get(product_url)

    zip_url_per_code = []
    for zipfile in r.html.find('a'):
        if ("zip" in zipfile.attrs['href']) and ("REFERENCE_MAP" not in zipfile.attrs['href']):
            zip_url_per_code.append("https://emergency.copernicus.eu"+zipfile.attrs['href'])
    zip_urls.append(zip_url_per_code)


KeyboardInterrupt: 

In [ ]:
code


In [82]:
len(zip_urls)

188

In [83]:
zip_urls

[['https://emergency.copernicus.eu/mapping/download/184631/EMSR501_AOI01_DEL_MONIT02_r1_RTP01_v1_vector.zip',
  'https://emergency.copernicus.eu/mapping/download/184632/EMSR501_AOI01_DEL_MONIT02_r1_VECTORS_v1_vector.zip',
  'https://emergency.copernicus.eu/mapping/download/184619/EMSR501_AOI01_DEL_MONIT01_r1_RTP01_v1_vector.zip',
  'https://emergency.copernicus.eu/mapping/download/184615/EMSR501_AOI01_DEL_MONIT01_r1_VECTORS_v1_vector.zip',
  'https://emergency.copernicus.eu/mapping/download/184610/EMSR501_AOI01_DEL_PRODUCT_r1_RTP01_v1_vector.zip',
  'https://emergency.copernicus.eu/mapping/download/184606/EMSR501_AOI01_DEL_PRODUCT_r1_VECTORS_v1_vector.zip'],
 ['https://emergency.copernicus.eu/mapping/download/184631/EMSR501_AOI01_DEL_MONIT02_r1_RTP01_v1_vector.zip',
  'https://emergency.copernicus.eu/mapping/download/184632/EMSR501_AOI01_DEL_MONIT02_r1_VECTORS_v1_vector.zip',
  'https://emergency.copernicus.eu/mapping/download/184619/EMSR501_AOI01_DEL_MONIT01_r1_RTP01_v1_vector.zip',
 

['https://emergency.copernicus.eu/mapping/download/184631/EMSR501_AOI01_DEL_MONIT02_r1_RTP01_v1_vector.zip',
 'https://emergency.copernicus.eu/mapping/download/184632/EMSR501_AOI01_DEL_MONIT02_r1_VECTORS_v1_vector.zip',
 'https://emergency.copernicus.eu/mapping/download/184619/EMSR501_AOI01_DEL_MONIT01_r1_RTP01_v1_vector.zip',
 'https://emergency.copernicus.eu/mapping/download/184615/EMSR501_AOI01_DEL_MONIT01_r1_VECTORS_v1_vector.zip',
 'https://emergency.copernicus.eu/mapping/download/184610/EMSR501_AOI01_DEL_PRODUCT_r1_RTP01_v1_vector.zip',
 'https://emergency.copernicus.eu/mapping/download/184606/EMSR501_AOI01_DEL_PRODUCT_r1_VECTORS_v1_vector.zip']

In [73]:
zip_urls

[<Element 'a' href='/mapping/download/184631/EMSR501_AOI01_DEL_MONIT02_r1_RTP01_v1_vector.zip'>,
 <Element 'a' href='/mapping/download/184632/EMSR501_AOI01_DEL_MONIT02_r1_VECTORS_v1_vector.zip'>,
 <Element 'a' href='/mapping/download/184619/EMSR501_AOI01_DEL_MONIT01_r1_RTP01_v1_vector.zip'>,
 <Element 'a' href='/mapping/download/184615/EMSR501_AOI01_DEL_MONIT01_r1_VECTORS_v1_vector.zip'>,
 <Element 'a' href='/mapping/download/184610/EMSR501_AOI01_DEL_PRODUCT_r1_RTP01_v1_vector.zip'>,
 <Element 'a' href='/mapping/download/184606/EMSR501_AOI01_DEL_PRODUCT_r1_VECTORS_v1_vector.zip'>,
 <Element 'a' href='/mapping/download/184631/EMSR501_AOI01_DEL_MONIT02_r1_RTP01_v1_vector.zip'>,
 <Element 'a' href='/mapping/download/184632/EMSR501_AOI01_DEL_MONIT02_r1_VECTORS_v1_vector.zip'>,
 <Element 'a' href='/mapping/download/184619/EMSR501_AOI01_DEL_MONIT01_r1_RTP01_v1_vector.zip'>,
 <Element 'a' href='/mapping/download/184615/EMSR501_AOI01_DEL_MONIT01_r1_VECTORS_v1_vector.zip'>,
 <Element 'a' href='

In [55]:
ems_tag = tf.Code[0]
session = HTMLSession()
product_url = "https://emergency.copernicus.eu/mapping/list-of-components/" + ems_tag

r = session.get(product_url)


In [74]:
zip_urls = []
for e in r.html.find("a"):
    if ("zip" in e.attrs["href"]) and ("REFERENCE_MAP" not in e.attrs["href"]):
        zip_urls.append(e.attrs['href'])

/mapping/download/184631/EMSR501_AOI01_DEL_MONIT02_r1_RTP01_v1_vector.zip
/mapping/download/184632/EMSR501_AOI01_DEL_MONIT02_r1_VECTORS_v1_vector.zip
/mapping/download/184619/EMSR501_AOI01_DEL_MONIT01_r1_RTP01_v1_vector.zip
/mapping/download/184615/EMSR501_AOI01_DEL_MONIT01_r1_VECTORS_v1_vector.zip
/mapping/download/184610/EMSR501_AOI01_DEL_PRODUCT_r1_RTP01_v1_vector.zip
/mapping/download/184606/EMSR501_AOI01_DEL_PRODUCT_r1_VECTORS_v1_vector.zip


In [30]:
#if __name__ == "__main__":
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s')

ems_web_page = "https://emergency.copernicus.eu/mapping/list-of-activations-rapid"
tables_floods = fetch_copernicusEMS(ems_web_page, ["Flood", "Storm"], "2021-01-06")


session = HTMLSession()
for i, t in enumerate(tables_floods.itertuples()):
    
    ems_tag = t._2
    logging.info("(%d/%d) Downloading files for event %s"%((i+1), tables_floods.shape[0], ems_tag))
    # logs the the files (each row) in table and prints the ems_tag with it

    product_url = "https://emergency.copernicus.eu/mapping/list-of-components/" + ems_tag
    r = session.get(product_url)

    a_elements = [e for e in r.html.find("a") if ("zip" in e.attrs["href"]) and ("REFERENCE_MAP" not in e.attrs["href"])]

    for j, e in enumerate(a_elements):
        zipfile_url = "https://emergency.copernicus.eu" + e.attrs["href"]
        print(zipfile_url)
        logging.info("\t (%d/%d) Downloading file %s"%((j+1), len(a_elements), zipfile_url))
        #download_vector_cems(zipfile_url)
        
        folder_out="CopernicusEMS"
        name_zip = os.path.basename(zipfile_url)
        #/ml4cc/worldfloods_sandbox/floodmaps_preprocessing/copernicusEMS/
        download_path = 'home/ahmedn/ml4cc/worldfloods_sandbox/floodmaps_preprocessing/copernicusEMS'
        download_vector_cems(zipfile_url, download_path, folder_out)
        #url_zip_file, download_path, folder_out="CopernicusEMS"

        if not os.path.exists(folder_out):
            os.mkdir(folder_out) # creates a directory called CopernicusEMS to hold data

        file_path_out = os.path.join(folder_out, name_zip)
        if os.path.exists(file_path_out):
            logging.info("\tFile %s exists will not be download"%zipfile_url)
            #return

        if is_downloadable(zipfile_url):
            r = requests.get(zipfile_url, allow_redirects=True)
            open(file_path_out, 'wb').write(r.content)
        if not os.path.exists(file_path_out):
            logging.warning("\tFAIL in download file %s" % zipfile_url)       

        

2021-02-15 21:25:52,911 root         INFO     (1/5) Downloading files for event EMSR501
2021-02-15 21:25:56,003 root         INFO     	 (1/6) Downloading file https://emergency.copernicus.eu/mapping/download/184631/EMSR501_AOI01_DEL_MONIT02_r1_RTP01_v1_vector.zip


https://emergency.copernicus.eu/mapping/download/184631/EMSR501_AOI01_DEL_MONIT02_r1_RTP01_v1_vector.zip


PermissionError: [Errno 1] Operation not permitted: '/usr/local/lib/python3.8/dist-packages/webbot/drivers/chrome_linux'

In [31]:
data  = {"confirmation": 1,
         "op":  "Download file",
         "form_build_id": "xxxx",
         "form_id": "emsmapping_disclaimer_download_form"}
# path_1 = "https://emergency.copernicus.eu/mapping/download/184472/EMSR498_AOI01_DEL_PRODUCT_r1_RTP02_v1_vector.zip"
path_2 = "https://emergency.copernicus.eu/mapping/download/184631/EMSR501_AOI01_DEL_MONIT02_r1_RTP01_v1_vector.zip "
r = requests.post(path_2,
                 allow_redirects=True, data=data)
r.headers["content-type"]
open("something_4.zip", 'wb').write(r.content)

18773545

In [ ]:
def unzip_copernicus_ems(data_root="."):
    """
    This function requires the data to be stored in the directory data_root
    with the file directory structure set by the worldfloods google cloud
    bucket. CopernicusEMS is a zip file that containes 

    """
    folder_out = os.path.join(data_root, "WORLDFLOODS/raw/CopernicusEMS/unzip")
    if not os.path.exists(folder_out):
        os.mkdir(folder_out)
    for f in glob(os.path.join(data_root, "WORLDFLOODS/raw/CopernicusEMS/raw/*.zip")):
        zip_ref = zipfile.ZipFile(f, 'r')
        directory_to_extract_to = os.path.join(folder_out, os.path.basename(os.path.splitext(f)[0]))
        if not os.path.exists(directory_to_extract_to):
            os.mkdir(directory_to_extract_to)

        zip_ref.extractall(directory_to_extract_to)
        zip_ref.close()

        # remove underscores
        for fextracted in glob(os.path.join(directory_to_extract_to,"*.*")):
            if " " not in fextracted:
                continue
            newname = fextracted.replace(" ","_")
            shutil.move(fextracted, newname)



formats = ["%d/%m/%Y T%H:%M:%SZ",
           "%d/%m/%Y %H:%M:%SZ",
           "%d/%m/%Y T%H:%M:%S",
           "%d/%m/%Y T%H:%MZ",
           "%d/%m/%Y T%H:%M UTC",
           "%d/%m/%Y %H:%M UTC",
           "%d/%m/%Y %H:%M", "%d/%m/%Y %H:%M:%S",
           "%Y/%m/%d %H:%M UTC",
           "%d/%m/%Y %H:%M:%S UTC"]

In [ ]:
def parse_date_messy(date_list):
    for fm in formats:
        try:
            date_list[0] = date_list[0].replace("119/09", "19/09").replace("05/24", "24/05")
            time_part = date_list[1].replace(".", ":").replace(";", ":").replace("UCT", "UTC")
            if time_part == "Not Applicable":
                time_part = "00:00"
            date_post_event = datetime.datetime.strptime(date_list[0] + ' ' + time_part, fm)
            if date_post_event.tzname() is None:
                date_post_event = date_post_event.replace(tzinfo=datetime.timezone.utc)
            return date_post_event
        except ValueError:
            pass

    return None

In [ ]:
def load_source_file(source_file, filter_event_dates=True, verbose=False):
    pd_source = Dbf5(source_file).to_dataframe()

    if "eventphase" not in pd_source.columns or not np.any(pd_source.eventphase == "Post-event"):
        if verbose:
            print("\t eventphase not found or not Post-event in source file")
        return

    dates_formated = []
    for d in pd_source.itertuples():
        date = parse_date_messy([d.src_date, d.source_tm])
        dates_formated.append(date)
    pd_source["date"] = dates_formated
    if filter_event_dates:
        pd_source = pd_source[(pd_source.eventphase == "Post-event") | (pd_source.eventphase == "Pre-event")]

        if np.any(pd.isna(pd_source.date)):
            if verbose:
                print("\t There are event phase files with undefined dates")

            pd_source = pd_source[~pd.isna(pd_source.date)]

        if not np.any(pd_source.eventphase == "Post-event"):
            if verbose:
                print("\t Post-event removed after filtering")
            return

    return pd_source

In [ ]:
def filter_register_copernicusems(source_file, tables_floods, verbose=False):
    observed_event_file = source_file.replace("_source.dbf", "_observed_event_a.shp")
    if not os.path.exists(observed_event_file):
        return

    product_name = os.path.basename(os.path.dirname(source_file))
    pd_source = load_source_file(source_file)
    if pd_source is None:
        return
    ems_code = product_name[:7]

    date_post_event = min(pd_source[pd_source.eventphase == "Post-event"]["date"])
    max_date_post_event = max(pd_source[pd_source.eventphase == "Post-event"]["date"])
    satellite_post_event = np.array(pd_source[(pd_source.eventphase == "Post-event") &
                                              (pd_source.date == date_post_event)]["source_nam"])[0]
    date_ems_code = datetime.datetime.strptime(tables_floods.loc[ems_code].loc["CodeDate"],
                                               "%Y-%m-%d").replace(tzinfo=datetime.timezone.utc)

    diff_dates = date_post_event - date_ems_code

    if diff_dates.days < 0:
        if verbose:
            print("Different between dates is negative %d" % diff_dates.days)
        return

    if diff_dates.days > 35:
        if verbose:
            print("difference too big %d" % diff_dates.days)
        return

    if (max_date_post_event-date_post_event).days >= 10:
        if verbose:
            print("difference between max date post event and min date post event too big %d" % (max_date_post_event-date_post_event).days)
        return

    stuff_pre_event = {}
    if np.any(pd_source.eventphase == "Pre-event"):
        date_pre_event = max(np.array(pd_source[pd_source.eventphase == "Pre-event"]["date"]))
        satellite_pre_event = np.array(pd_source[(pd_source.eventphase == "Pre-event") &
                                                 (pd_source.date == date_pre_event)]["source_nam"])[0]

        stuff_pre_event["satellite_pre_event"] = satellite_pre_event
        stuff_pre_event["timestamp_pre_event"] = date_pre_event
        if (date_post_event - date_pre_event).days < 0:
            if verbose:
                print("Date pre event %s is after date post event %s" % (date_pre_event.strftime("%Y-%m-%d"),
                                                                         date_post_event.strftime("%Y-%m-%d")))
            return

    # Filter content of shapefile
    pd_geo = geopandas.read_file(observed_event_file)
    if np.any(pd_geo["event_type"] != '5-Flood'):
        if verbose:
            print("%s Event type is not Flood"%observed_event_file)
        return

    if not isinstance(satellite_post_event, str):
        if verbose:
            print("Satellite post event: {} is not a string!".format(satellite_post_event))
        return

    register = {"name": product_name,
                "ems_code": ems_code,
                "timestamp": date_post_event,
                "satellite": satellite_post_event,
                "timestamp_ems_code": date_ems_code,
                "observed_event_file": observed_event_file}

    # Add hydrology file
    hidrology_file = source_file.replace("_source.dbf", "_hydrography_a.shp")
    if os.path.exists(hidrology_file):
        register["hydrology_file"] = hidrology_file

    register.update(stuff_pre_event)
    return register


In [ ]:
def filter_copernicusems(tables_floods, verbose=False, data_root="."):
    files_sentinel = []
    for f in sorted(glob(os.path.join(data_root,
                                      "WORLDFLOODS/raw/CopernicusEMS/unzip/*/*_source.dbf"))):
        register = filter_register_copernicusems(f, tables_floods, verbose=verbose)
        if register is not None:
            files_sentinel.append(register)

    return files_sentinel

In [ ]:
def copy_to_bucket(bucket, files_to_copy, folder_event):
    for fc in files_to_copy:
        ext = os.path.splitext(fc)[1]
        blob_name = os.path.join(folder_event, "map" + ext)
        if bucket.get_blob(blob_name) is None:
            blob = bucket.blob(blob_name)
            blob.upload_from_filename(fc)

In [ ]:
def get_bbox(pd_geo):
    bounds = pd_geo.bounds
    minx = np.min(bounds.minx)
    miny = np.min(bounds.miny)
    maxx = np.max(bounds.maxx)
    maxy = np.max(bounds.maxy)

    return {"west": minx, "east": maxx, "north": maxy, "south": miny}

In [ ]:
def processing_register(register, folder, filename, bucket):
    layer_name = os.path.join("WORLDFLOODS/maps/", folder, os.path.splitext(os.path.basename(filename.replace(" ", "_")))[0])
    files_to_copy = [f for f in glob(os.path.splitext(filename)[0] + ".*") if
                     "kmz" not in f[-4::]]
    copy_to_bucket(bucket, files_to_copy, layer_name)
    pd_geo = geopandas.read_file(filename)
    bounding_box = get_bbox(pd_geo)

    if "obj_desc" in pd_geo:
        event_type = np.unique(pd_geo.obj_desc)
        if len(event_type) > 1:
            print("Multiple event types within shapefile {}".format(event_type))
        event_type = event_type[0]
    else:
        event_type = "NaN"

    crs_code_space, crs_code = pd_geo.crs["init"].split(":")
    meta = {
        'event id': register["name"],
        'layer name': os.path.basename(layer_name),
        'event type': event_type,
        'satellite date': register["timestamp"].isoformat(),
        'country': "NaN",
        'satellite': register["satellite"],
        'bounding box': bounding_box,
        'reference system': {
            'code space': crs_code_space,
            'code': crs_code
        },
        'abstract': "NaN",
        'purpose': "NaN",
        'source': 'CopernicusEMS'
    }
    blob_name = os.path.join(layer_name, "meta.json")
    if bucket.get_blob(blob_name) is None:
        with open("meta.json", "w") as fh:
            json.dump(meta, fh)
        blob = bucket.blob(blob_name)
        blob.upload_from_filename("meta.json")

In [ ]:
def copy_clean_data_to_bucket(files_copernicus_ems):
    bucket = storage.Client().get_bucket("fdl_floods_2019_data")

    print("Copy files to filtered bucket")

    for _i,register in enumerate(files_copernicus_ems):
        print("Uploading file %d/%d %s" % (_i + 1, len(files_copernicus_ems), register["name"]))
        # Copy flood
        processing_register(register, "flood", register["observed_event_file"], bucket)

        # Copy hydrology
        if "hydrology_file" in register:
            processing_register(register, "hydrography", register["hydrology_file"], bucket)


if __name__ == "__main__":
    data_ems = table_floods_ems()
    print("%d flood events in Copernicus EMS service" % (data_ems.shape[0]))

    # Download the data from the bucket
    # ! mkdir -p WORLDFLOODS/raw/CopernicusEMS/raw
    # ! gsutil -m cp -r gs://fdl_floods_2019_data/WORLDFLOODS/raw/CopernicusEMS/raw/ WORLDFLOODS/raw/CopernicusEMS/raw

    # unzip_copernicus_ems()
    files_sen = filter_copernicusems(data_ems)

    print("%d files in clean dataset" % len(files_sen))

    copy_clean_data_to_bucket(files_sen)


In [9]:
url_zip_file

NameError: name 'url_zip_file' is not defined

In [6]:
import requests
test = requests.post('https://emergency.copernicus.eu/mapping/download/184472/EMSR498_AOI01_DEL_PRODUCT_r1_RTP02_v1_vector.zip')

In [7]:
test

<Response [200]>

In [9]:
tables_floods.tail()

,Unnamed: 0,Act. Code,Title,Event Date,Type,Country/Terr.,Feed
229,NaN,EMSR258,Flood in Albania,2017-12-01,Flood,Albania,NaN
230,NaN,EMSR257,"Flood in Attika, Greece",2017-11-15,Flood,Greece,NaN
232,NaN,EMSR255,"Wind Storm in Saxony, Germany",2017-10-29,Storm,Germany,NaN
236,NaN,EMSR251,Floods in Norway,2017-10-21,Flood,Norway,NaN
238,NaN,EMSR249,Hurricane Ophelia in Ireland,2017-10-16,Storm,Ireland,NaN


In [15]:
for i, j in enumerate(tables_floods.itertuples()):
    print(i, j)

0 Pandas(Index=1, _1=nan, _2='EMSR498', Title='Flood in Corrèze department, France', _4='2021-02-02', Type='Flood', _6='France', Feed=nan)
1 Pandas(Index=2, _1=nan, _2='EMSR497', Title='Flood in Germany', _4='2021-02-01', Type='Flood', _6='Germany', Feed=nan)
2 Pandas(Index=3, _1=nan, _2='EMSR496', Title='Flood in Lazio Region, Italy', _4='2021-01-26', Type='Flood', _6='Italy', Feed=nan)
3 Pandas(Index=4, _1=nan, _2='EMSR495', Title='Tropical cyclone Eloise in Mozambique,...', _4='2021-01-22', Type='Storm', _6='Mozambique, Eswatini, Zimbabwe', Feed=nan)
4 Pandas(Index=7, _1=nan, _2='EMSR492', Title='Flood in Landes, France', _4='2021-01-01', Type='Flood', _6='France', Feed=nan)
5 Pandas(Index=9, _1=nan, _2='EMSR489', Title='Tropical Cyclone Yasa in Fiji', _4='2020-12-17', Type='Storm', _6='Fiji', Feed=nan)
6 Pandas(Index=10, _1=nan, _2='EMSR488', Title='Flood in Lazio Region, Italy', _4='2020-12-08', Type='Flood', _6='Italy', Feed=nan)
7 Pandas(Index=11, _1=nan, _2='EMSR487', Title='Fl

In [53]:
tables_floods.iloc[0]

Unnamed: 0                                       NaN
Act. Code                                    EMSR498
Title            Flood in Corrèze department, France
Event Date                                2021-02-02
Type                                           Flood
Country/Terr.                                 France
Feed                                             NaN
Name: 1, dtype: object

In [37]:
t

Pandas(Index=238, _1=nan, _2='EMSR249', Title='Hurricane Ophelia in Ireland', _4='2017-10-16', Type='Storm', _6='Ireland', Feed=nan)

In [ ]:
product_url

In [16]:
ems_web_page

'https://emergency.copernicus.eu/mapping/list-of-activations-rapid'

In [17]:
tables = pd.read_html(ems_web_page)

In [19]:
tables[0]

,Unnamed: 0,Act. Code,Title,Event Date,Type,Country/Terr.,Feed
0,NaN,EMSR500,Wildfire in Western Australia,2021-02-05,Wildfire,Australia,NaN
1,NaN,EMSR498,"Flood in Corrèze department, France",2021-02-02,Flood,France,NaN
2,NaN,EMSR497,Flood in Germany,2021-02-01,Flood,Germany,NaN
3,NaN,EMSR496,"Flood in Lazio Region, Italy",2021-01-26,Flood,Italy,NaN
4,NaN,EMSR495,"Tropical cyclone Eloise in Mozambique,...",2021-01-22,Storm,"Mozambique, Eswatini, Zimbabwe",NaN
...,...,...,...,...,...,...,...
450,NaN,EMSR006,Blast of arms depot in Bulgaria,2012-06-05,Industrial acc...,Bulgaria,NaN
451,NaN,EMSR005,Earthquake in Western Bulgaria,2012-05-22,Earthquake,Bulgaria,NaN
452,NaN,EMSR004,Earthquake in Emilia Romagna,2012-05-20,Earthquake,Italy,NaN
453,NaN,EMSR003,Forest Fire in Bács-Kiskun,2012-04-29,"Forest fire, w...",Hungary,NaN


In [24]:
r = session.get(product_url)

In [25]:
r

<Response [200]>

In [10]:
ems_web_page = "https://emergency.copernicus.eu/mapping/list-of-activations-rapid"
tables = pd.read_html(ems_web_page)[0] 
# pd.read_html gives back a single item list
tables_floods = tables[(tables.Type == "Flood") | (tables.Type == "Storm")]
tables_floods = tables_floods[tables_floods["Event Date"] >= "2017-10-01"]


In [11]:
mystringlist = ["Flood", "Storm"]

In [25]:
tables_floods

,Unnamed: 0,Act. Code,Title,Event Date,Type,Country/Terr.,Feed
1,NaN,EMSR498,"Flood in Corrèze department, France",2021-02-02,Flood,France,NaN
2,NaN,EMSR497,Flood in Germany,2021-02-01,Flood,Germany,NaN
3,NaN,EMSR496,"Flood in Lazio Region, Italy",2021-01-26,Flood,Italy,NaN
4,NaN,EMSR495,"Tropical cyclone Eloise in Mozambique,...",2021-01-22,Storm,"Mozambique, Eswatini, Zimbabwe",NaN
7,NaN,EMSR492,"Flood in Landes, France",2021-01-01,Flood,France,NaN
...,...,...,...,...,...,...,...
229,NaN,EMSR258,Flood in Albania,2017-12-01,Flood,Albania,NaN
230,NaN,EMSR257,"Flood in Attika, Greece",2017-11-15,Flood,Greece,NaN
232,NaN,EMSR255,"Wind Storm in Saxony, Germany",2017-10-29,Storm,Germany,NaN
236,NaN,EMSR251,Floods in Norway,2017-10-21,Flood,Norway,NaN


In [39]:
tables_fl = tables[tables["Type"].isin(mystringlist)]# & tables["Event Date"] >= "2017-10-01"]
tables_fl = tables_fl[tables_fl["Event Date"] >= "2017-10-01"]

In [37]:
tables_fl.shape

(130, 7)

In [23]:
tables_floods.shape

(130, 7)

In [27]:
tables_floods.head()

,Unnamed: 0,Act. Code,Title,Event Date,Type,Country/Terr.,Feed
1,NaN,EMSR498,"Flood in Corrèze department, France",2021-02-02,Flood,France,NaN
2,NaN,EMSR497,Flood in Germany,2021-02-01,Flood,Germany,NaN
3,NaN,EMSR496,"Flood in Lazio Region, Italy",2021-01-26,Flood,Italy,NaN
4,NaN,EMSR495,"Tropical cyclone Eloise in Mozambique,...",2021-01-22,Storm,"Mozambique, Eswatini, Zimbabwe",NaN
7,NaN,EMSR492,"Flood in Landes, France",2021-01-01,Flood,France,NaN


In [28]:
tables_fl.head()

,Unnamed: 0,Act. Code,Title,Event Date,Type,Country/Terr.,Feed
1,NaN,EMSR498,"Flood in Corrèze department, France",2021-02-02,Flood,France,NaN
2,NaN,EMSR497,Flood in Germany,2021-02-01,Flood,Germany,NaN
3,NaN,EMSR496,"Flood in Lazio Region, Italy",2021-01-26,Flood,Italy,NaN
4,NaN,EMSR495,"Tropical cyclone Eloise in Mozambique,...",2021-01-22,Storm,"Mozambique, Eswatini, Zimbabwe",NaN
7,NaN,EMSR492,"Flood in Landes, France",2021-01-01,Flood,France,NaN


In [30]:
tables_fl

,Unnamed: 0,Act. Code,Title,Event Date,Type,Country/Terr.,Feed
1,NaN,EMSR498,"Flood in Corrèze department, France",2021-02-02,Flood,France,NaN
2,NaN,EMSR497,Flood in Germany,2021-02-01,Flood,Germany,NaN
3,NaN,EMSR496,"Flood in Lazio Region, Italy",2021-01-26,Flood,Italy,NaN
4,NaN,EMSR495,"Tropical cyclone Eloise in Mozambique,...",2021-01-22,Storm,"Mozambique, Eswatini, Zimbabwe",NaN
7,NaN,EMSR492,"Flood in Landes, France",2021-01-01,Flood,France,NaN
...,...,...,...,...,...,...,...
436,NaN,EMSR020,Floods in Slovenia,2012-11-05,Flood,Slovenia,NaN
437,NaN,EMSR019,Floods in Cameroon,2012-09-06,Flood,Cameroon,NaN
438,NaN,EMSR018,Floods in Niger,2012-08-24,Flood,Niger,NaN
439,NaN,EMSR017,Floods in Philippines,2012-08-06,Flood,Philippines,NaN


In [31]:
tables_floods

,Unnamed: 0,Act. Code,Title,Event Date,Type,Country/Terr.,Feed
1,NaN,EMSR498,"Flood in Corrèze department, France",2021-02-02,Flood,France,NaN
2,NaN,EMSR497,Flood in Germany,2021-02-01,Flood,Germany,NaN
3,NaN,EMSR496,"Flood in Lazio Region, Italy",2021-01-26,Flood,Italy,NaN
4,NaN,EMSR495,"Tropical cyclone Eloise in Mozambique,...",2021-01-22,Storm,"Mozambique, Eswatini, Zimbabwe",NaN
7,NaN,EMSR492,"Flood in Landes, France",2021-01-01,Flood,France,NaN
...,...,...,...,...,...,...,...
229,NaN,EMSR258,Flood in Albania,2017-12-01,Flood,Albania,NaN
230,NaN,EMSR257,"Flood in Attika, Greece",2017-11-15,Flood,Greece,NaN
232,NaN,EMSR255,"Wind Storm in Saxony, Germany",2017-10-29,Storm,Germany,NaN
236,NaN,EMSR251,Floods in Norway,2017-10-21,Flood,Norway,NaN
